In [1]:
import pandas as pd
import numpy as np
import urllib
import requests
from bs4 import BeautifulSoup
import os
from time import sleep
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from time import sleep
import random

In [2]:
df = pd.read_csv("links_folha_sem_rotulos.csv")
df.shape


(3622, 2)

In [4]:
df.head()

,ano,link
0,2001,http://www1.folha.uol.com.br/fsp/ilustrad/fq04...
1,2001,http://www1.folha.uol.com.br/folha/mundo/ult94...
2,2001,http://www1.folha.uol.com.br/fsp/ilustrad/fq10...
3,2001,http://www1.folha.uol.com.br/folha/ciencia/ult...
4,2001,http://www1.folha.uol.com.br/folha/ciencia/ult...


In [2]:
import spacy
nlp = spacy.load("pt_core_news_sm")

In [3]:
def limpa_paragrafos(link, tamanho):
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"}
    try:
        req = Request(link, headers = headers)
        response = urlopen(req)
       
    except HTTPError as e: # erro de segurança do site, pode ser necessário usar user-Agent
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
#     html = urllib.request.urlopen(link).read()
    html = response.read()
    soup = BeautifulSoup(html)

    for script in soup(["script", "style"]):
        script.decompose()

    lista = []
    for i in soup.stripped_strings:
        lista.append(i)
    
#     print(lista)

    miolo = []
    for i in lista:
        if len(i) > tamanho:
            miolo.append(i.replace('\r\n',''))
#             print(miolo)

    paragrafos = []
    for i in range(len(miolo)):
        if ('esquizo' in miolo[i]) or ('Esquizo' in miolo[i]):
            paragrafos.append(miolo[i])
    
   
    return paragrafos, miolo

In [7]:
link = df.link.values[3]
link

'http://www1.folha.uol.com.br/folha/ciencia/ult306u2066.shtml'

In [8]:
paragrafos, miolo = limpa_paragrafos(link, 70)

In [9]:
paragrafos

['Folha Online - Ciência - Israelenses descobrem gene que indica propensão à esquizofrenia - 16/01/2001 23h02',
 'Um grupo de pesquisadores israelenses descobriu um gene que pode servir para diagnosticar a esquizofrenia via exame de sangue. O gene, conhecido como D3, está ligado à produção da dopamina, substância envolvida na transmissão de impulsos nervosos.',
 'A descoberta pode trazer benefício aos pacientes porque reduziria o tempo de testes antes do início de um tratamento. Normalmente, a esquizofrenia é diagnosticada no início da idade adulta. A doença não tem cura e as drogas usadas no tratamento causam graves efeitos colaterais.',
 '"A dopamina é o principal neurotransmissor do sistema nervoso central e seus receptores estão associados a diversos distúrbios como a doença de Parkinson e a esquizofrenia", afirmou Tal Ilani, que coordena uma equipe do Instituto de Ciência Weizmann, em Rehotov, Israel. Um artigo sobre o estudo foi publicado pela Academia Nacional de Ciências dos EU

In [10]:
miolo

['Folha Online - Ciência - Israelenses descobrem gene que indica propensão à esquizofrenia - 16/01/2001 23h02',
 'Um grupo de pesquisadores israelenses descobriu um gene que pode servir para diagnosticar a esquizofrenia via exame de sangue. O gene, conhecido como D3, está ligado à produção da dopamina, substância envolvida na transmissão de impulsos nervosos.',
 'A descoberta pode trazer benefício aos pacientes porque reduziria o tempo de testes antes do início de um tratamento. Normalmente, a esquizofrenia é diagnosticada no início da idade adulta. A doença não tem cura e as drogas usadas no tratamento causam graves efeitos colaterais.',
 '"A dopamina é o principal neurotransmissor do sistema nervoso central e seus receptores estão associados a diversos distúrbios como a doença de Parkinson e a esquizofrenia", afirmou Tal Ilani, que coordena uma equipe do Instituto de Ciência Weizmann, em Rehotov, Israel. Um artigo sobre o estudo foi publicado pela Academia Nacional de Ciências dos EU

In [4]:
df = pd.read_csv("RESGATE_PARCIAL_CONTINUAR.csv")
df.shape

(3622, 4)

In [6]:
df[df.texto_completo.isnull()]

,ano,link,paragrafos_resgatados,texto_completo
2775,2015,http://www1.folha.uol.com.br/ilustrada/2015/08...,NaN,NaN
2776,2015,http://www1.folha.uol.com.br/fsp/ilustrada/230...,NaN,NaN
2777,2015,http://www1.folha.uol.com.br/livrariadafolha/2...,NaN,NaN
2778,2015,http://www1.folha.uol.com.br/fsp/cotidiano/224...,NaN,NaN
2779,2015,http://www1.folha.uol.com.br/cotidiano/2015/07...,NaN,NaN
...,...,...,...,...
3617,2019,https://www1.folha.uol.com.br/poder/2019/10/ev...,NaN,NaN
3618,2020,https://www1.folha.uol.com.br/opiniao/2020/01/...,NaN,NaN
3619,2021,https://www1.folha.uol.com.br/poder/2021/03/pr...,NaN,NaN
3620,2021,https://f5.folha.uol.com.br/televisao/2021/01/...,NaN,NaN


In [7]:
n_links = df.shape[0]
n_links

3622

In [8]:
for i in range(2774,n_links):

    print(f"Processando {i} de {n_links}.")

    link = df.link.iloc[i]
    paragrafos, miolo = limpa_paragrafos(link, 70)
    df.loc[i, "paragrafos_resgatados"] = str(paragrafos)
    df.loc[i, "texto_completo"] = str(miolo)

    numero_randomico = random.randint(1, 10)
    sleep(numero_randomico)

Processando 2774 de 3622.
Processando 2775 de 3622.
Processando 2776 de 3622.
Processando 2777 de 3622.
Processando 2778 de 3622.
Processando 2779 de 3622.
Processando 2780 de 3622.
Processando 2781 de 3622.
Processando 2782 de 3622.
Processando 2783 de 3622.
Processando 2784 de 3622.
Processando 2785 de 3622.
Processando 2786 de 3622.
Processando 2787 de 3622.
Processando 2788 de 3622.
Processando 2789 de 3622.
Processando 2790 de 3622.
Processando 2791 de 3622.
Processando 2792 de 3622.
Processando 2793 de 3622.
Processando 2794 de 3622.
Processando 2795 de 3622.
Processando 2796 de 3622.
Processando 2797 de 3622.
Processando 2798 de 3622.
Processando 2799 de 3622.
Processando 2800 de 3622.
Processando 2801 de 3622.
Processando 2802 de 3622.
Processando 2803 de 3622.
Processando 2804 de 3622.
Processando 2805 de 3622.
Processando 2806 de 3622.
Processando 2807 de 3622.
Processando 2808 de 3622.
Processando 2809 de 3622.
Processando 2810 de 3622.
Processando 2811 de 3622.
Processando 

In [9]:
df[df.paragrafos_resgatados.notnull()]

,ano,link,paragrafos_resgatados,texto_completo
0,2001,http://www1.folha.uol.com.br/fsp/ilustrad/fq04...,['É uma idiotice. Mas é o preço do sucesso dos...,['Folha de S.Paulo - Contardo Calligaris: Tamp...
1,2001,http://www1.folha.uol.com.br/folha/mundo/ult94...,"['Os transtornos psíquicos, inclusive as manif...",['Folha Online - Mundo - OMS diz quer 400 milh...
2,2001,http://www1.folha.uol.com.br/fsp/ilustrad/fq10...,['O artista plástico Ubirajara Ferreira Braga ...,"['Ubirajara Ferreira Braga, ex-interno do Juqu..."
3,2001,http://www1.folha.uol.com.br/folha/ciencia/ult...,['Folha Online - Ciência - Israelenses descobr...,['Folha Online - Ciência - Israelenses descobr...
4,2001,http://www1.folha.uol.com.br/folha/ciencia/ult...,['Folha Online - Ciência - Pesquisadores anunc...,['Folha Online - Ciência - Pesquisadores anunc...
...,...,...,...,...
3617,2019,https://www1.folha.uol.com.br/poder/2019/10/ev...,"['""O advogado e ex-ministro Admar e\xa0a advog...","[""Evento da PM em SP tem vaias a Doria e grito..."
3618,2020,https://www1.folha.uol.com.br/opiniao/2020/01/...,['Tanta pressa gera reações esquizofrênicas. V...,['Este conteúdo é para maiores de 18 anos. Se ...
3619,2021,https://www1.folha.uol.com.br/poder/2021/03/pr...,"['Mas os professores do ódio, aqueles que baju...",['Este conteúdo é para maiores de 18 anos. Se ...
3620,2021,https://f5.folha.uol.com.br/televisao/2021/01/...,"[""Emilio Boechat diz que mudanças da Igreja Un...","[""Emilio Boechat diz que mudanças da Igreja Un..."


In [11]:
df.to_csv("links_folha_sem_rotulo_resgatados.csv", index = False)

In [13]:
df[df.paragrafos_resgatados.isnull()]

,ano,link,paragrafos_resgatados,texto_completo


In [14]:
df_vazios = df.query("paragrafos_resgatados == '[]'")
df_vazios

,ano,link,paragrafos_resgatados,texto_completo
28,2001,http://www1.folha.uol.com.br/folha/brasil/ult9...,[],['Folha Online - Brasil - Controle do comércio...
32,2001,http://www1.folha.uol.com.br/fsp/ciencia/fe110...,[],['Folha de S.Paulo - Saiba mais: Retrovírus es...
80,2001,http://www1.folha.uol.com.br/fsp/opiniao/fz011...,[],"['""O atentado à igreja católica em Behawalpur,..."
90,2001,http://www1.folha.uol.com.br/fsp/ciencia/fe101...,[],['Folha de S.Paulo - Pesquisa: Rejeição a tran...
95,2001,http://www1.folha.uol.com.br/fsp/cotidian/ff30...,[],['Folha de S.Paulo - Estradas: Duplicação da R...
...,...,...,...,...
3535,2015,http://www1.folha.uol.com.br/livrariadafolha/2...,[],['Compartilhar Governo Lula: Comandante do Exé...
3553,2018,http://www1.folha.uol.com.br/livrariadafolha/2...,[],['Compartilhar Governo Lula: Comandante do Exé...
3565,2021,https://entretempos.blogfolha.uol.com.br/?p=22475,[],['Entretempos – Artes visuais diluídas em dife...
3566,2021,https://viradapsicodelica.blogfolha.uol.com.br...,[],['Virada Psicodélica – Novidades da fronteira ...


In [15]:
df_vazios.to_csv("links_folha_sem_rotulo_resgatados_vazios.csv", index = False)